In [1]:
import numpy as np
import pandas as pd
import math

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB2 #260x260
import tensorflow.keras.layers as layers
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.utils import Sequence
import imgaug.augmenters as iaa
import imgaug

from sklearn.model_selection import train_test_split

2023-01-12 20:16:13.408822: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_dir = '/run/user/1000/gvfs/smb-share:server=titan.local,share=datasets/kitchenware-classification'
seed = 42

In [3]:
df_train_full = pd.read_csv(data_dir+'/train.csv', dtype={'Id': str})
df_extra = pd.read_csv(data_dir+'/data.csv', dtype={'Id': str})
df_train_full = pd.concat([df_train_full, df_extra], ignore_index = True)
df_train_full['filename'] = data_dir+'/images/' + df_train_full['Id'] + '.jpg'
df_train_full.head()

,Id,label,filename
0,0560,glass,/run/user/1000/gvfs/smb-share:server=titan.loc...
1,4675,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...
2,0875,glass,/run/user/1000/gvfs/smb-share:server=titan.loc...
3,4436,spoon,/run/user/1000/gvfs/smb-share:server=titan.loc...
4,8265,plate,/run/user/1000/gvfs/smb-share:server=titan.loc...


In [4]:
image_width = 260
image_height = 260
NUM_CLASSES = 6
batch_size = 32
n_epochs = 10

In [5]:
df_train, df_val, y_train, y_val = train_test_split(df_train_full, df_train_full['label'], test_size = 0.2, shuffle = True, stratify = df_train_full['label'], random_state = seed)
df_train

,Id,label,filename
7592,eu.95a129ef-d94a-4ab0-a4e4-e66aac61e55f,fork,/run/user/1000/gvfs/smb-share:server=titan.loc...
7704,eu.b90f170d-c2d2-47bf-bd61-375f9032e98f,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...
7840,eu.84a95cb4-6026-40ed-bfb6-936d297a7d91,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...
5030,5485,fork,/run/user/1000/gvfs/smb-share:server=titan.loc...
7114,eu.edfa5512-6f5d-4cf7-bbcc-a31eace9b484,spoon,/run/user/1000/gvfs/smb-share:server=titan.loc...
...,...,...,...
5033,5725,plate,/run/user/1000/gvfs/smb-share:server=titan.loc...
4172,5120,plate,/run/user/1000/gvfs/smb-share:server=titan.loc...
4507,9013,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...
3625,2070,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...


In [6]:
train_aug = iaa.Sequential(
    [
        iaa.Resize({"height": image_height, "width": image_width}, interpolation=imgaug.ALL),
        iaa.Fliplr(0.3),
        iaa.Flipud(0.2), # vertically flip 20% of all images
        # Make some images brighter and some darker.
        # In 20% of all cases, we sample the multiplier once per channel,
        # which can end up changing the color of the images.
        iaa.Sometimes(0.3,iaa.Multiply((0.8, 1.2), per_channel=0.2)),
        # `Sometimes()` applies a function randomly to the inputs with
        # a given probability (0.3, in this case).
        iaa.Sometimes(0.3, iaa.Cutout()),
        iaa.Sometimes(0.1, iaa.GaussianBlur(sigma=(0, 0.5))),
        iaa.Sometimes(0.1, iaa.Cartoon(blur_ksize=3, segmentation_size=1.0, saturation=2.0, edge_prevalence=1.0))
    ],
    name = "train_aug"
)

In [7]:
class KitchenwareSequence(Sequence):
    """
    def __init__(self, x_in, y_in, aug=None, batch_size, shuffle=True):
        # Initialization
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.x = x_in
        self.y = y_in
        self.aug = aug
        self.datalen = len(y_in)
        self.indexes = np.arange(self.datalen)
        if self.shuffle:
            np.random.shuffle(self.indexes)
    """
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size


    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        print(np.array(batch_x))
        return np.array(batch_x), np.array(batch_y)
    
    def __len__(self):
        # Denotes the number of batches per epoch
        return math.ceil(len(self.x) / self.batch_size)

    """
    def on_epoch_end(self):
        # Updates indexes after each epoch
        self.indexes = np.arange(self.datalen)
        if self.shuffle:
            np.random.shuffle(self.indexes)
    """

In [8]:
"""
model = EfficientNetB2(
    weights='imagenet',
    include_top=False,
    input_shape=(image_width, image_height, 3)
)
model.trainable = False

inputs = keras.Input(shape=(image_width, image_height, 3))

base = model(inputs, training=False)
vectors = keras.layers.GlobalAveragePooling2D()(base)
outputs = keras.layers.Dense(6)(vectors)

model = keras.Model(inputs, outputs)

learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

loss = keras.losses.CategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
"""

"\nmodel = EfficientNetB2(\n    weights='imagenet',\n    include_top=False,\n    input_shape=(image_width, image_height, 3)\n)\nmodel.trainable = False\n\ninputs = keras.Input(shape=(image_width, image_height, 3))\n\nbase = model(inputs, training=False)\nvectors = keras.layers.GlobalAveragePooling2D()(base)\noutputs = keras.layers.Dense(6)(vectors)\n\nmodel = keras.Model(inputs, outputs)\n\nlearning_rate = 0.01\noptimizer = keras.optimizers.Adam(learning_rate=learning_rate)\n\nloss = keras.losses.CategoricalCrossentropy(from_logits=True)\n\nmodel.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])\n"

In [9]:
def build_model(num_classes):
    
    inputs = layers.Input(shape = (image_width, image_height, 3))
    #x = train_aug(inputs)
    x=inputs
    #outputs = EfficientNetB2(include_top=False, weights='imagenet', classes=NUM_CLASSES)(x)
    #model = tf.keras.Model(inputs, outputs)
    model = EfficientNetB2(include_top=False, input_tensor=x, weights="imagenet")
    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    
    return model

In [10]:
#train = KitchenwareSequence(df_train, df_train['label'], batch_size=batch_size)
#val = KitchenwareSequence(df_val, df_val['label'], batch_size=batch_size)

In [11]:
#ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
#ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))

In [12]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='label',
    target_size=(image_width, image_height),
    batch_size=32,
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_generator = val_datagen.flow_from_dataframe(
    df_val,
    x_col='filename',
    y_col='label',
    target_size=(image_width, image_height),
    batch_size=32,
)

Found 6390 validated image filenames belonging to 6 classes.
Found 1598 validated image filenames belonging to 6 classes.


In [14]:
model = build_model(num_classes=NUM_CLASSES)

#epochs = 25  # @param {type: "slider", min:8, max:80}
#hist = model.fit(train, epochs=n_epochs, validation_data=val, verbose=2)
history = model.fit(
    train_generator,
    epochs=2,
    validation_data=val_generator
)
#plot_hist(history)


Epoch 1/2
200/200 [==============================] - 128s 620ms/step - loss: 0.5729 - accuracy: 0.9124 - val_loss: 0.3368 - val_accuracy: 0.9318
Epoch 2/2
200/200 [==============================] - 124s 623ms/step - loss: 0.4940 - accuracy: 0.9360 - val_loss: 0.5121 - val_accuracy: 0.9393


In [16]:
df_test = pd.read_csv(data_dir+'/test.csv', dtype={'Id': str})
df_test['filename'] = data_dir+'/images/' + df_test['Id'] + '.jpg'
df_test.head()

,Id,filename
0,0678,/run/user/1000/gvfs/smb-share:server=titan.loc...
1,3962,/run/user/1000/gvfs/smb-share:server=titan.loc...
2,9271,/run/user/1000/gvfs/smb-share:server=titan.loc...
3,5133,/run/user/1000/gvfs/smb-share:server=titan.loc...
4,8842,/run/user/1000/gvfs/smb-share:server=titan.loc...


In [17]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_dataframe(
    df_test,
    x_col='filename',
    class_mode='input',
    target_size=(image_width, image_height),
    batch_size=32,
    shuffle=False
)

Found 3808 validated image filenames.


In [18]:
y_pred = model.predict(test_generator)

119/119 [==============================] - 119s 994ms/step


In [20]:
classes = np.array(list(train_generator.class_indices.keys()))
classes

array(['cup', 'fork', 'glass', 'knife', 'plate', 'spoon'], dtype='<U5')

In [21]:
predictions = classes[y_pred.argmax(axis=1)]

In [22]:
df_submission = pd.DataFrame()
df_submission['filename'] = test_generator.filenames
df_submission['label'] = predictions

df_submission['Id'] = df_submission.filename.str[len(data_dir+'/images/'):-4]
del df_submission['filename']

In [23]:
df_submission[['Id', 'label']].to_csv('submissions/effb2_imnet.csv', index=False)

In [25]:
!kaggle competitions submit kitchenware-classification -f submissions/effb2_imnet.csv -m 'validation: 0.9393'

100%|██████████████████████████████████████| 38.6k/38.6k [00:01<00:00, 22.5kB/s]
Successfully submitted to Kitchenware Classification